# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности

In [125]:
# Зависимости кода, указанного в задании

import re
import textdistance
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

with open('wiki_data.txt', encoding='utf-8') as f:
    corpus = f.read()
vocab = Counter(re.findall(r'\w+', corpus.lower()))
id2word = {i:word for i, word in enumerate(vocab)}
vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [126]:
# Код, указанный в задании

def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    
    return closest

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [127]:
# Код решения задания

import numpy as np

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn * 4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    closest_with_probabilities = [(word[0], word[1], np.log(P(word[0]))) for word in closest]
    # добавляем вероятности к результату

    return sorted(closest_with_probabilities, key=lambda x: (x[1], x[2]), reverse=True)
    # сортируем сначала по расстоянию редактирования, затем по вероятности

get_closest_hybrid_match('паровощ', X, vec, 20)
# среди кандидатов с одинаковым расстоянием редактирования наиболее вероятные всегда будут первыми

[('паровоз', 0.8571428571428572, -11.60538419447003),
 ('паровой', 0.8571428571428572, -11.900183734690675),
 ('паровом', 0.8571428571428572, -14.356919507511979),
 ('паровое', 0.8571428571428572, -14.762384615620144),
 ('паровоза', 0.75, -12.236655971311889),
 ('паровозы', 0.75, -12.236655971311889),
 ('парового', 0.75, -13.846093883745988),
 ('паровозу', 0.75, -14.069237435060199),
 ('парковое', 0.75, -14.762384615620144),
 ('парчовое', 0.75, -15.455531796180088),
 ('паровая', 0.7142857142857143, -12.565160038283924),
 ('паровые', 0.7142857142857143, -12.682943073940308),
 ('паровую', 0.7142857142857143, -13.25830721884387),
 ('паров', 0.7142857142857143, -13.376090254500253),
 ('шаровой', 0.7142857142857143, -13.509621647124774),
 ('карово', 0.7142857142857143, -14.762384615620144),
 ('паромов', 0.7142857142857143, -14.762384615620144),
 ('даровой', 0.7142857142857143, -15.455531796180088),
 ('парво', 0.7142857142857143, -15.455531796180088),
 ('паровым', 0.7142857142857143, -15.455

In [128]:
# или если нужно только слово:
get_closest_hybrid_match('паровощ', X, vec, 20)[0][0]

'паровоз'

## 2. Symspell

In [129]:
import re
from collections import Counter, defaultdict

# Составляем словарь правильных слов
with open('wiki_data.txt', encoding='utf-8') as f:
    corpus = f.read()
vocab = Counter(re.findall(r'\w+', corpus.lower()))

In [130]:
# Составляем словарь удалений
deletions_vocab = defaultdict(list)
for word in vocab:
    for index, _ in enumerate(word):
        deletions_vocab[word[0:index] + word[index + 1:]].append(word)

In [131]:
def symspell(bad_word: str, deletions: dict, main_vocab: dict) -> str:
    if bad_word in main_vocab:
        return bad_word
    candidates = set()
    for i, _ in enumerate(bad_word):
        candidates.update(deletions[bad_word[0:i] + bad_word[i + 1:]])
    candidates = sorted(candidates, key=lambda x: main_vocab[x], reverse=True) # сортируем кандидатов по вероятности
    try:
        return candidates[0]
    except IndexError:
        return bad_word

In [132]:
symspell('паровощ', deletions_vocab, vocab)

'паровоз'

In [133]:
from string import punctuation

bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [134]:
# Оценка качества
def evaluate(correction):
    correct = 0
    total = 0

    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    cashed = {}
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            # чтобы два раза не исправлять одно и тоже слово - закешируем его
            # перед тем как считать исправление проверим нет ли его в кеше

            predicted = cashed.get(pair[1], correction(pair[1], deletions_vocab, vocab))
            cashed[pair[1]] = predicted


            if predicted == pair[0]:
                correct += 1
            total += 1

            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] !=  predicted:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == predicted:
                    mistaken_fixed += 1
    return {'correct':correct, 'total':total, 'mistaken_fixed': mistaken_fixed, 'total_mistaken':total_mistaken, 
            'correct_broken': correct_broken, 'total_correct':total_correct}

In [135]:
result = evaluate(symspell)
print(result['correct']/result['total'])
print(result['mistaken_fixed']/result['total_mistaken'])
print(result['correct_broken']/result['total_correct'])

0.855927963981991
0.19875776397515527
0.04685884920179166


Результат не очень хороший - исправляются только 20% ошибок. Можно попробовать увеличить процент, если перед генерацией удалений записать в список кандидатов варианты исправлений для самого ошибочного слова в неизменном виде:

In [136]:
def symspell_modified(bad_word: str, deletions: dict, main_vocab: dict) -> str:
    if bad_word in main_vocab:
        return bad_word
    candidates = set(deletions[bad_word]) # это делается здесь
    for i, _ in enumerate(bad_word):
        candidates.update(deletions[bad_word[0:i] + bad_word[i + 1:]])
    candidates = sorted(candidates, key=lambda x: main_vocab[x], reverse=True) # сортируем кандидатов по вероятности
    try:
        return candidates[0]
    except IndexError:
        return bad_word

In [137]:
result = evaluate(symspell_modified)
print(result['correct']/result['total'])
print(result['mistaken_fixed']/result['total_mistaken'])
print(result['correct_broken']/result['total_correct'])

0.8680340170085042
0.30667701863354035
0.04892615137245894


Уже лучше, но всё равно только 30%. Так как в чистом виде в алгоритме создаётся только словарь удалений, вряд ли удастся серьёзно улучшить результат без значительных изменений алгоритма